# Analysis on output of the simulation VEG_SHIFT_IDEAL

In [1]:
import numpy as np
import netCDF4
import pandas as pd
import xarray as xr
import glob #return all file paths that match a specific pattern
import matplotlib as mpl
import matplotlib.pyplot as plt

import sys; sys.path.append("..")
from dataset_manipulation import fix_cam_time

**NOTE**: If you then copy this to NIRD remember to change the path of the input files and the libraries

In [ ]:
# Import 
raw_path = '../../../archive/' #Betzy: /cluster/home/adelez/nird/ #Nird: /nird/home/adelez/storage/
processed_path = '../../processed-data/postprocessing/'

casename = 'VEG_SHIFT_IDEAL_2000_sec_nudg_f19_f19'
casealias = 'IDEAL-ON'

fp = raw_path+casename+'/atm/hist/'+casename+'.cam.h0.*.nc'
#VEG_SHIFT_IDEAL_2000_sec_nudg_f19_f19.cam.h0.2007-01.nc
all_files = glob.glob(fp)
all_files.sort()
print("Files found")

ds = xr.open_mfdataset(all_files)
print("Dataset created")

Files found


In [ ]:
ds

In [ ]:
# Fix timestamp of model data
ds = fix_cam_time(ds)

# Remove spinup months of data set
ds = ds.isel(time=slice(12,len(ds.time)))

print("Postprocessing completed")
date = '20082012'

In [ ]:
# BVOC variables
variables = ['SFisoprene', 'SFmonoterp']
ds[variables].to_netcdf(processed_path+casealias+'_'+'BVOC_'+date+'.nc')

In [ ]:
"""Av = 6.022e23  # Avogadro's number
M_iso = 68.114200e-3  # Molar mass isoprene kg/mol
M_mono = 136.228400e-3  # Molar mass monoterpene kg/mol
path_this_file = str(pathlib.Path().absolute())

var_dic = dict(SFmonoterp='H10H16',
            SFisoprene='ISOP')
M_dic = dict(SFmonoterp=M_mono,
            SFisoprene=M_iso)"""

In [ ]:
# SOA variables
variables = ['N_AER', 'SOA_A1','SOA_NA','cb_SOA_A1','cb_SOA_NA', 'cb_SOA_A1_OCW', 'cb_SOA_NA_OCW']
ds[variables].to_netcdf(processed_path+casealias+'_'+'SOA_'+date+'.nc')

In [ ]:
# CLOUD PROPERTIES
variables = ['ACTNL', 'ACTREL','CDNUMC', 'CLDHGH', 'CLDLOW', 'CLDMED', 'CLDTOT', 'CLDLIQ', 'CLOUD', 'CLOUDCOVER_CLUBB', 'FCTL', 'LWCF', 'SWCF', 'NUMLIQ', 'TGCLDLWP']
ds[variables].to_netcdf(processed_path+casealias+'_'+'CLOUDPROP_'+date+'.nc')

In [ ]:
# RADIATIVE COMPONENTS
variables = ['FLNT', 'FSNT', 'FLNT_DRF', 'FLNTCDRF', 'FSNTCDRF', 'FSNT_DRF']

xr_ds = ds.copy()
xr_ds = xr_ds[variables]

for var in ['SWDIR_Ghan', 'LWDIR_Ghan', 'DIR_Ghan', 'SWCF_Ghan', 'LWCF_Ghan', 'NCFT_Ghan', 'SW_rest_Ghan', 'LW_rest_Ghan']:
    
    if ('SWDIR_Ghan' == var):# or ('DIR_Ghan' == var):
        xr_ds['SWDIR_Ghan'] = xr_ds['FSNT'] - xr_ds['FSNT_DRF']
        xr_ds['SWDIR_Ghan'].attrs['units'] = xr_ds['FSNT_DRF'].attrs['units']
    if ('LWDIR_Ghan' == var):# or ('DIR_Ghan' == var):
        xr_ds['LWDIR_Ghan'] = -(xr_ds['FLNT'] - xr_ds['FLNT_DRF'])
        xr_ds['LWDIR_Ghan'].attrs['units'] = xr_ds['FLNT_DRF'].attrs['units']
    if 'DIR_Ghan' == var:
        xr_ds['DIR_Ghan'] = xr_ds['LWDIR_Ghan'] + xr_ds['SWDIR_Ghan']
        xr_ds['DIR_Ghan'].attrs['units'] = xr_ds['LWDIR_Ghan'].attrs['units']
    if 'SWCF_Ghan' == var:
        xr_ds['SWCF_Ghan'] = xr_ds['FSNT_DRF'] - xr_ds['FSNTCDRF']
        xr_ds[var].attrs['units'] = xr_ds['FSNT_DRF'].attrs['units']
    if 'LWCF_Ghan' == var:
        xr_ds[var] = -(xr_ds['FLNT_DRF'] - xr_ds['FLNTCDRF'])
        xr_ds[var].attrs['units'] = xr_ds['FLNT_DRF'].attrs['units']
    if 'NCFT_Ghan' == var:
        xr_ds[var] = xr_ds['FSNT_DRF'] - xr_ds['FSNTCDRF'] - (xr_ds['FLNT_DRF'] - xr_ds['FLNTCDRF'])
        xr_ds[var].attrs['units'] = xr_ds['FLNT_DRF'].attrs['units']
    if 'SW_rest_Ghan' == var:
        xr_ds[var] = xr_ds['FSNTCDRF']
    elif 'LW_rest_Ghan' == var:
        xr_ds[var] = xr_ds['FLNTCDRF']
    
xr_ds.to_netcdf(processed_path+casealias+'_'+'RADIATIVE_'+date+'.nc')

In [10]:
# TURBULENT FLUXES
variables = ['LHFLX', 'OMEGAT', 'SHFLX']
ds[variables].to_netcdf(processed_path+casealias+'_'+'TURBFLUXES_'+date+'.nc')

In [9]:
import glob
import xarray as xr
from datetime import datetime

# List all matching files
fns = 'VEG_SHIFT_IDEAL_2000_sec_nudg_f19_f19.cam.h0.*.nc'
files = glob.glob(fn_path+fns)

# Create list for 
individual_files = []

# Loop through each file in the list
for i in files:
    
    # Load a single dataset
    #timestep_ds = xr.open_dataset(i)
    timestep_ds = xr.open_mfdataset(i)
    
    # Create a new variable called 'time' from the `time_coverage_start` field, and 
    # convert the string to a datetime object so xarray knows it is time data
    timestep_ds['time'] = datetime.strptime(timestep_ds.time_coverage_start, 
                                           "%Y-%m-%dT%H:%M:%S.%fZ")
    
    # Add the dataset to the list
    individual_files.append(timestep_ds)

# Combine individual datasets into a single xarray along the 'time' dimension
modis_ds = xr.concat(individual_files, dim='time')

print(modis_ds)

ModuleNotFoundError: No module named 'dask'